In [1]:
import pandas as pd
import numpy as np
import warnings
import scipy.stats as stats
from scipy.integrate import quad
from statsmodels.stats.multitest import multipletests
import pickle as pkl
warnings.filterwarnings('ignore')

In [2]:
# Dictionary for keeping DFE datasets: {DFE_study: {environment: DataFrame[f'{env}_fitness', f'{env}_P']}}
DFE_dict = {}

In [3]:
# Load DFEs of 21 yeast genes

DFE_dict_tmp = {}

Gene_list = ['ADA2','PRS3','ASC1','RAD6','BFR1','RPL29', \
             'BUD23','RPL39','CCW12','RPS7A','EOS1','SNF6','GET1', \
             'TSR2','GIM5','VMA21','IES6','VMA7','LSM1','EST1','PAF1']
df_YPD = pd.DataFrame()

# Concatenate per-gene fitness tables, drop nonsense mutations
for Gene in Gene_list:
    df_gene = pd.read_csv(f'./data/Fitness_landscapes/21_genes_in_yeast/{Gene}.txt', 
                     delim_whitespace=True)
    df_gene['Gene'] = Gene
    df_YPD = pd.concat([df_YPD,df_gene],ignore_index=True)
df_YPD = df_YPD[df_YPD['Mutation_type'] != 'Nonsense_mutation']

# Compute mean fitness across YPD replicates and test if it is significantly different from 1 (non-neutral)
df_YPD['YPD_fitness'] = df_YPD[[f'Fitness_from_YPD_replicate_{i}' for i in range(1,5)]].mean(axis=1)
_,P = stats.ttest_1samp(df_YPD[['Fitness_from_YPD_replicate_1','Fitness_from_YPD_replicate_2',
           'Fitness_from_YPD_replicate_3','Fitness_from_YPD_replicate_4']],1,axis=1)
df_YPD['YPD_P'] = P

# Store DFE (fitness, P) for nonsynonymous mutations only
DFE_dict_tmp['YPD'] = \
    df_YPD.loc[df_YPD['Mutation_type'] == 'Nonsynonymous_mutation',['YPD_fitness','YPD_P']]

# DFE in SC_37, YPD_H2O2, and YPE environments, drop nonsense mutations
df_all = pd.read_csv('./data/Fitness_landscapes/21_genes_in_yeast/All_mutations_4env_19genes.csv')
df_all = df_all[df_all['Mutation_type'] != 'Nonsense_mutation']

# Compute mean fitness across replicates and test if it is significantly different from 1 (non-neutral)
for env in ['SC_37','YPD_H2O2','YPE']:
    df_all[f'{env}_fitness'] = df_all[[f'Fitness_from_{env}_replicate_{i}' for i in range(1,4)]].mean(axis=1)
    _,P = stats.ttest_1samp(df_all[[f'Fitness_from_{env}_replicate_{i}' for i in range(1,4)]],1,axis=1)
    df_all[f'{env}_P'] = P

    # SNF6 gene in YPE environment has only 2 replicate
    if env == 'YPE':
        _,P_SNF6 = stats.ttest_1samp(
            df_all.loc[df_all['Gene'] == 'SNF6', [f'Fitness_from_{env}_replicate_{i}' for i in range(1,3)]],
            1,axis=1
        )
        df_all.loc[df_all['Gene'] == 'SNF6', f'{env}_P'] = P_SNF6

    # Store DFE (fitness, P) for nonsynonymous mutations only
    DFE_dict_tmp[env] = \
        df_all.loc[df_all['Mutation_type'] == 'Nonsynonymous_mutation', [f'{env}_fitness',f'{env}_P']]

# Add to master dict
DFE_dict['Yeast21genes'] = DFE_dict_tmp

In [4]:
# Load DFEs of HSP82

DFE_dict_tmp = {}
hsp_df = pd.read_csv('./data/Fitness_landscapes/HSP82/HSP82_FL.csv')

# Screen single nonsynonymous mutations
idx = (hsp_df['mut_type'] == 'Nonsynonymous') & (hsp_df['N_mut'] == 1)

# Extract (fitness, P) information for each environment
for env in ['standard_merged','diamide','ethanol','nitrogen_depletion','salt','37C']:
    tmp_df = hsp_df.loc[idx,[f'fitness_{env}',f'P_{env}']]
    tmp_df = tmp_df.rename(columns={f'fitness_{env}':f'{env}_fitness',f'P_{env}':f'{env}_P'})
    DFE_dict_tmp[env] = tmp_df
    
# Add to master dict
DFE_dict['HSP82'] = DFE_dict_tmp

In [5]:
# Load DFEs of UBI4

DFE_dict_tmp = {}
ubi_df = pd.read_csv('./data/Fitness_landscapes/UBI4/UBI4_fitness.csv')
env = 'standard'

# Screen single nonsynonymous mutations, and extract (fitness, P) information
tmp_df = \
    ubi_df.loc[(ubi_df['mut_type'] == 'Nonsynonymous') & (ubi_df['N_mut'] == 1),['fitness','P']]
tmp_df = tmp_df.rename(columns={f'fitness':f'{env}_fitness',f'P':f'{env}_P'})
DFE_dict_tmp[env] = tmp_df

# Add to master dict
DFE_dict['UBI4'] = DFE_dict_tmp

In [7]:
# Load DFEs of folA

DFE_dict_tmp = {}

# Extract (fitness, P) information for nonsynonymous mutations in each environment
for env in ['no_Lon','Lon']:
    result_df = pd.read_csv(f'./data/Fitness_landscapes/folA/folA_{env}_result.csv')
    # Convert growth rate to fitness
    result_df['fitness'] = np.exp(result_df['growth_rate']*(1/0.23))
    idx_non = result_df['mut_type'] == 'Nonsynonymous'
    idx_notna = result_df['fitness'].notna()
    tmp_df = result_df.loc[idx_non & idx_notna,['fitness','P']]
    tmp_df = tmp_df.rename(columns={f'fitness':f'{env}_fitness',f'P':f'{env}_P'})
    DFE_dict_tmp[env] = tmp_df

# Add to master dict
DFE_dict['folA'] = DFE_dict_tmp


In [9]:
def downsample_ben(df_rep, env):
    """
    Construct downsampled DFEs with beneficial mutations occuring 10 times less often than the original.
    Input:
        df_rep: original DFE DataFrame to be sampled from; env: environment name
    Returns:
        (df_combined, df_combined_sig) : DataFrames of resampled DFE, 
        without and with requiring beneficial mutations to be statistical significant, respectively.
    """
    N_sample = 10000
    idx_ben = df_rep[f'{env}_fitness'] > 1
    idx_sig = df_rep[f'{env}_P'] <= 0.05

    # Get frequencies of beneficial mutations without and with statistical significance,
    # mutations with missing fitness effect are excluded
    F_ben = idx_ben.sum()/df_rep[f'{env}_fitness'].notna().sum()
    F_ben_sig = (idx_ben & idx_sig).sum()/(df_rep[f'{env}_fitness'].notna()).sum()

    # Set number of sampled beneficial muataitons to be 10% of that in the original DFE
    N_ben = int(N_sample*F_ben*0.1)
    N_nonben = N_sample - N_ben

    # Resample the new DFE
    df_ben = df_rep[idx_ben].sample(N_ben,replace=True)
    df_nonben = df_rep[df_rep[f'{env}_fitness']<=1].sample(N_nonben,replace=True)
    df_combined = pd.concat([df_nonben,df_ben])

    # For the "significant only" version, set all nonsignificant fitness effects to neutral (1.0)
    df_tmp = df_rep.copy()
    idx = df_tmp[f'{env}_P'] > 0.05
    df_tmp.loc[idx,f'{env}_fitness'] = 1

    # Set number of sampled beneficial muataitons to be 10% of that in the original DFE, and resample the new DFE
    N_ben_sig = int(N_sample*F_ben_sig*0.1)
    N_nonben_sig = N_sample - N_ben_sig
    df_ben_sig = df_tmp[idx_ben & idx_sig].sample(N_ben_sig,replace=True)
    df_nonben_sig = df_tmp[df_tmp[f'{env}_fitness']<=1].sample(N_nonben_sig,replace=True)
    df_combined_sig = pd.concat([df_nonben_sig,df_ben_sig])
    
    return df_combined, df_combined_sig

In [10]:
def p_to_q(p_series):
    """
    Convert P-values (NA allowed) to q-values using Benjamini–Hochberg (BH) step-up procedure,
    preserving the original index and NaNs.
    """
    valid_pvals = p_series.dropna()
    _, qvals, _, _ = multipletests(valid_pvals, method='fdr_bh')
    qvals_series = pd.Series(np.nan, index=p_series.index)
    qvals_series[valid_pvals.index] = qvals
    return qvals_series

In [11]:
def get_fix_prob(fitness_list, Ne):
    """
    Calculate fixation probability given the fitness effect and effective population size.
    For small |s| < 1/(2Ne), use the approximation fix_prob = 1/(2Ne) to avoid numerical issues.
    """
    s = fitness_list - 1
    fix_prob = (1 - np.exp(-2 * s)) / (1 - np.exp(-4 * Ne * s))
    idx = (np.abs(s) < 1 / (2 * Ne)) | (s == 0)
    fix_prob[idx] = 1 / (2 * Ne)
    return fix_prob

def cal_omega(fitness_list, Ne):
    """
    Calculate dN/d_netural (Omega) given the fitness effect and effective population size:
    """
    fix_prob = get_fix_prob(fitness_list, Ne)
    Omega = fix_prob.mean() * 2 * Ne
    return Omega

def cal_alpha(fitness_list, Ne):
    """
    Calculate fraction of beneficial substitutions among all substitutions (Alpha)
    given the fitness effect and effective population size:
    """
    fix_prob = get_fix_prob(fitness_list, Ne)
    benefit_idx = fitness_list - 1 > 1 / (2 * Ne)
    Alpha = fix_prob[~benefit_idx].sum() / fix_prob.sum()
    return Alpha

In [12]:
def L(s, a, u, Ne):
    """
    Lambda function for clonal interference model as described in (Gerrish & Lenski, 1998)
    s: selection coefficient (>0)
    a: 1 / mean(s_beneficial)
    u: beneficial mutation rate
    Ne: effective population size
    Returns the expected number of interfering mutations.
    """
    return u/s*Ne*np.log(Ne)*np.exp(-a*s)*2*(s+1/a)

def mean_fixP(a, u, Ne):
    """
    Calculate mean colonal-interference-adjusted fixation probability
    across exponential DFE of beneficial mutations as described in (Gerrish & Lenski, 1998)
    Integrates over s ~ Exp(rate=a) from 0 to +inf.
    Returns (value, error).
    """
    P_CI, error = quad(lambda x: a*2*x*np.exp(-L(x,a,u,Ne)-a*x), 0, np.inf)
    return P_CI

In [13]:
def cal_asexual(df_rep,env,Ne,u=1.2e-4):
    """
    Calculate Omega and Alpha with asexual colonal interference.
    with a clonal-interference average for clearly beneficial ones.
      - Estimate exponential distribution parameter a = 1 / mean(s) over s > 1/(2Ne).
      - Compute P_CI under exponential tail with rate 'a'.
      - Use P_Kimura elsewhere; replace beneficial entries' P_fix with P_CI.
    Returns:
        (Omega, Alpha) under asexual colonal interference adjustion.
    """
    s = df_rep[f'{env}_fitness'] - 1
    
    # Exponential tail parameter from observed beneficial s
    a = 1 / s[s > 1 / (2 * Ne)].mean()
    
    # Mean fixation probability for beneficial mutations considering CI
    P_CI = mean_fixP(a, u, Ne) 
    # Mean fixation probability using Kimura formula
    P_Kimura = (1 - np.exp(-2 * s)) / (1 - np.exp(-4 * Ne * s))
    # Override fix prob of beneficial mutaions using CI-adjusted fix prob
    P_fix = P_Kimura
    P_fix[s > 1 / (2 * Ne)] = P_CI
    
    # Aggregate metrics
    Omega = P_fix.mean() * (2 * Ne)
    Alpha = P_fix[s < 1 / (2 * Ne)].sum() / P_fix.sum()
    return Omega, Alpha

In [14]:
# Main loop: compute Omega and Alpha across Ne, DFEs, environments
alpha_Ne_dict = {}
omega_Ne_dict = {}

for Ne in [1e4, 1e6, 1e7, 1e8]:
    print(Ne)
    alpha_Ne_dict[f'{Ne:.0e}'] = {}
    omega_Ne_dict[f'{Ne:.0e}'] = {}

    # Loop through different DFEs;
    for gene in ['Yeast21genes','HSP82','UBI4','folA']:
        alpha_Ne_dict[f'{Ne:.0e}'][gene] = {}
        omega_Ne_dict[f'{Ne:.0e}'][gene] = {}
        print(gene)
        df_env_dict = DFE_dict[gene]
        for env,df_rep in df_env_dict.items():
            
            # Baseline Kimura estimates
            Omega = cal_omega(df_rep[f'{env}_fitness'], Ne)
            Alpha = cal_alpha(df_rep[f'{env}_fitness'], Ne)
            
            # Asexual CI-adjusted estimates
            Omega_asexual,Alpha_asexual = cal_asexual(df_rep,env,Ne,u=1.2e-4)

            # Significance-filtered (raw P > 0.05 set to neutral)
            df_tmp = df_rep.copy()
            idx = df_tmp[f'{env}_P'] > 0.05
            df_tmp.loc[idx,f'{env}_fitness'] = 1
            Omega_sig = cal_omega(df_tmp[f'{env}_fitness'], Ne)
            Alpha_sig = cal_alpha(df_tmp[f'{env}_fitness'], Ne)

            # Significance-filtered (q-value > 0.05 set to neutral)
            df_tmp = df_rep.copy()
            idx_FDR005 = p_to_q(df_tmp[f'{env}_P']) > 0.05
            df_tmp.loc[idx_FDR005, f'{env}_fitness'] = 1
            Omega_FDR005 = cal_omega(df_tmp[f'{env}_fitness'], Ne)
            Alpha_FDR005 = cal_alpha(df_tmp[f'{env}_fitness'], Ne)

            # Significance-filtered (q-value > 0.01 set to neutral)
            df_tmp = df_rep.copy()
            idx_FDR001 = p_to_q(df_tmp[f'{env}_P']) > 0.01
            df_tmp.loc[idx_FDR001, f'{env}_fitness'] = 1
            Omega_FDR001 = cal_omega(df_tmp[f'{env}_fitness'], Ne)
            Alpha_FDR001 = cal_alpha(df_tmp[f'{env}_fitness'], Ne)
            
            # Downsampling beneficial mutations by 10 fold
            Omega_ds_list = []
            Alpha_ds_list = []
            Omega_ds_sig_list = []
            Alpha_ds_sig_list = []
            Omega_ds_asexual_list = []
            Alpha_ds_asexual_list = []
            
            for i in range(100):
                # Get downsample DFEs
                df_ds_ben, df_ds_ben_sig = downsample_ben(df_rep, env)

                # Baseline Kimura estimates on downsampled
                Omega_ds = cal_omega(df_ds_ben[f'{env}_fitness'], Ne)
                Alpha_ds = cal_alpha(df_ds_ben[f'{env}_fitness'], Ne)

                #Significance-filtered (non-significant set to neutral) on downsampled
                Omega_ds_asexual,Alpha_ds_asexual = cal_asexual(df_ds_ben,env,Ne,u=1.2e-5)

                # Significance-filtered (non-significant set to neutral) on downsampled
                Omega_ds_sig = cal_omega(df_ds_ben_sig[f'{env}_fitness'], Ne)
                Alpha_ds_sig = cal_alpha(df_ds_ben_sig[f'{env}_fitness'], Ne)

                # Accumulate
                Omega_ds_list.append(Omega_ds)
                Alpha_ds_list.append(Alpha_ds)
                Omega_ds_sig_list.append(Omega_ds_sig)
                Alpha_ds_sig_list.append(Alpha_ds_sig)
                Omega_ds_asexual_list.append(Omega_ds_asexual)
                Alpha_ds_asexual_list.append(Alpha_ds_asexual)

            # Store results
            omega_Ne_dict[f'{Ne:.0e}'][gene][env] = [
                Omega, 
                Omega_sig, 
                Omega_FDR005,
                Omega_FDR001,
                Omega_asexual,
                np.mean(Omega_ds_list), 
                np.mean(Omega_ds_sig_list), 
                np.mean(Omega_ds_asexual_list)
            ]
            
            alpha_Ne_dict[f'{Ne:.0e}'][gene][env] = [
                Alpha, 
                Alpha_sig, 
                Alpha_FDR005, 
                Alpha_FDR001, 
                Alpha_asexual,
                np.mean(Alpha_ds_list), 
                np.mean(Alpha_ds_sig_list), 
                np.mean(Alpha_ds_asexual_list),
            ]


10000.0
Yeast21genes
HSP82
UBI4
folA
1000000.0
Yeast21genes
HSP82
UBI4
folA
10000000.0
Yeast21genes
HSP82
UBI4
folA
100000000.0
Yeast21genes
HSP82
UBI4
folA


In [ ]:
# Print selected results
for Ne in ['1e+06', '1e+07', '1e+08']:
    print(Ne)
    omega_gene_dict = omega_Ne_dict[Ne]
    alpha_gene_dict = alpha_Ne_dict[Ne]
    for gene in ['Yeast21genes']: # Modify to show results of other DFE.
        print(gene)
        omega_env_dict = omega_gene_dict[gene]
        alpha_env_dict = alpha_gene_dict[gene]
        for env,omega_list in omega_env_dict.items():
            alpha_list = alpha_env_dict[env]
            print(env)
            print(
               omega_list[0],omega_list[1],omega_list[4],omega_list[5],omega_list[6],omega_list[7],
               alpha_list[0],alpha_list[1],alpha_list[4],alpha_list[5],alpha_list[6],alpha_list[7],
               sep='\t'
            )

In [ ]:
# Save results.
# The pickles files have been generated in data folder.
# Uncomment and run this block if you want to overwrite.

# with open('./data/omega_Ne_dict.pkl','wb') as f:
#     pkl.dump(omega_Ne_dict,f)
# with open('./data/alpha_Ne_dict.pkl','wb') as f:
#     pkl.dump(alpha_Ne_dict,f)